# Span Batch Analysis - Data Preparation

## Initialization

Before start, Fill in [secret.yml](span-batch-analysis/secret.yaml). Refer to [secret_example.yml](span-batch-analysis/secret_example.yaml). This is for specifying RPC endpoints.

Also check config located at [config](span-batch-analysis/config/).

In [2]:
import os

Build binaries.

In [3]:
%%bash
cd cmd/batch_decoder && go build . && cd ../../
cd cmd/span_batch_tester && go build . && cd ../../

In [4]:
%%capture
%env BATCH_DECODER=cmd/batch_decoder/batch_decoder
%env SPAN_BATCH_TESTER=cmd/span_batch_tester/span_batch_tester

Clone superchain config.

In [5]:
%%bash
git clone https://github.com/ethereum-optimism/superchain-registry
cd superchain-registry && git checkout 5d3ca1966435827ac8c4216bddcb541ec64c1d6c && cd --

fatal: destination path 'superchain-registry' already exists and is not an empty directory.


HEAD is now at 5d3ca19 superchain/extra: include minified genesis state data (#2)


Select your chain.

In [6]:
%%capture
%env L1_CHAIN_NAME=mainnet
%env L2_CHAIN_NAME=pgn

# available networks
#      L1 - L2 list
# mainnet - base, op, pgn, zora
# goerli  - base, op, zora
# sepolia - op, pgn

In [7]:
l1_chain_name = os.environ["L1_CHAIN_NAME"]
l2_chain_name = os.environ["L2_CHAIN_NAME"]

start_number = !yq eval ".start_l1_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
start_number_value = int(start_number[0])
end_number = !yq eval ".end_l1_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
end_number_value = int(end_number[0])
batch_tx_sender = !yq eval ".batch_tx_sender" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
batch_tx_sender_value = batch_tx_sender[0]
l2_genesis_timestamp = !yq eval ".genesis.l2_time" "superchain-registry/superchain/configs/{l1_chain_name}/{l2_chain_name}.yaml"
l2_genesis_timestamp_value = int(l2_genesis_timestamp[0])
l2_chain_id = !yq eval ".chain_id" "superchain-registry/superchain/configs/{l1_chain_name}/{l2_chain_name}.yaml"
l2_chain_id_value = int(l2_chain_id[0])
inbox_addr = !yq eval ".batch_inbox_addr" "superchain-registry/superchain/configs/{l1_chain_name}/{l2_chain_name}.yaml"
inbox_addr_value = inbox_addr[0]

acc_start_number = !yq eval ".acc_l2_start_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
acc_start_number_value = int(acc_start_number[0])
acc_end_number = !yq eval ".acc_l2_end_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
acc_end_number_value = int(acc_end_number[0])

Read analyis config.

In [8]:
%env L1_START=$start_number_value
%env L1_END=$end_number_value
%env BATCH_TX_SENDER=$batch_tx_sender_value
%env L2_GENESIS_TIMESTAMP=$l2_genesis_timestamp_value
%env L2_CHAIN_ID=$l2_chain_id_value
%env INBOX=$inbox_addr_value

%env ACC_L2_START=$acc_start_number_value
%env ACC_L2_END=$acc_end_number_value


env: L1_START=18005000
env: L1_END=18025000
env: BATCH_TX_SENDER=0x99526b0e49a95833e734eb556a6abaffab0ee167
env: L2_GENESIS_TIMESTAMP=1689106727
env: L2_CHAIN_ID=424
env: INBOX=0xC1B90E1e459aBBDcEc4DCF90dA45ba077d83BFc5
env: ACC_L2_START=2142000
env: ACC_L2_END=2143000


Get L1 RPC from secret.

In [9]:
key_name = f"L1_{l1_chain_name}_RPC".upper()
L1_RPC = !yq eval ".{key_name}" span-batch-analysis/secret.yaml
L1_RPC_ENDPOINT = L1_RPC[0]

key_name = f"L2_{l2_chain_name}_{l1_chain_name}_RPC".upper()
L2_RPC = !yq eval ".{key_name}" span-batch-analysis/secret.yaml
L2_RPC_ENDPOINT = L2_RPC[0]

In [10]:
%%capture
%env L1_RPC=$L1_RPC_ENDPOINT
%env L2_RPC=$L2_RPC_ENDPOINT

## Data Collection

All data will be stored at `./artifacts/{L1_CHAIN_NAME}/{L2_CHAIN_NAME}`

In [11]:
l1_chain_name = os.environ["L1_CHAIN_NAME"]
l2_chain_name = os.environ["L2_CHAIN_NAME"]

%env DB_PREFIX=./artifacts/$l1_chain_name/$l2_chain_name

env: DB_PREFIX=./artifacts/mainnet/pgn


batch decoder `fetch`: Read channel frames from L1 by reading batch transactions.

In [11]:
!$BATCH_DECODER fetch --l1=$L1_RPC --start=$L1_START --end=$L1_END --inbox=$INBOX --out=$DB_PREFIX/transactions_cache --sender=$BATCH_TX_SENDER --concurrent-requests=25 | tail -10

Fetched block:  18024989
Fetched block:  18024996
Fetched block:  18024971
Fetched block:  18024998
Fetched block:  18024987
Fetched block:  18024981
Fetched block:  18024984
Fetched batches in range [18005000,18025000). Found 408 valid & 0 invalid batches
Fetch Config: Chain ID: 1. Inbox Address: 0xC1B90E1e459aBBDcEc4DCF90dA45ba077d83BFc5. Valid Senders: map[0x99526b0e49A95833E734EB556A6aBaFFAb0Ee167:{}].
Wrote transactions with batches to ./artifacts/mainnet/pgn/transactions_cache


batch decoder `reassemble`: Reassemble channel frames to channels. 

In [12]:
!$BATCH_DECODER reassemble --inbox=$INBOX --in=$DB_PREFIX/transactions_cache --out=$DB_PREFIX/channel_cache | tail -10

span batch tester `convert`: Convert channels with singular batches to span batches. Try multiple tx encoding types.

In [13]:
!TX_ENCODE_TYPE=0  && $SPAN_BATCH_TESTER convert --in=$DB_PREFIX/channel_cache --out=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --l2=$L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[399/408] Channel ID: f718760db35172a2224b731719ed6c76, L2StartNum: 2099776, L2EndNum: 2100073, L2BlockCnt, 298
[400/408] Channel ID: f8e001697b195b21e58a35126a52ec5b, L2StartNum: 2077784, L2EndNum: 2078085, L2BlockCnt, 302
[401/408] Channel ID: f9427495f0dbf967f238998c44011902, L2StartNum: 2124724, L2EndNum: 2125023, L2BlockCnt, 300
[402/408] Channel ID: f999b633449917a75d12a1236b697de0, L2StartNum: 2060554, L2EndNum: 2060845, L2BlockCnt, 292
[403/408] Channel ID: f9aa2069176be12edd4f34c6a863764b, L2StartNum: 2043099, L2EndNum: 2043390, L2BlockCnt, 292
[404/408] Channel ID: fae0d8a3afd0ae92befaebf716c3fd11, L2StartNum: 2070094, L2EndNum: 2070393, L2BlockCnt, 300
[405/408] Channel ID: fb44b33b51a977272b32d4fe0713f810, L2StartNum: 2087589, L2EndNum: 2087888, L2BlockCnt, 300
[406/408] Channel ID: fd22d73ca508673ef67453f79f518e7e, L2StartNum: 2014856, L2EndNum: 2015150, L2BlockCnt, 295
[407/408] Channel ID: fe37e62106e73584f7934c3c1201a6f5, L2StartNum: 2021379, L2EndNum: 2021678, L2BlockC

In [14]:
!TX_ENCODE_TYPE=1  && $SPAN_BATCH_TESTER convert --in=$DB_PREFIX/channel_cache --out=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --l2=$L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[399/408] Channel ID: f718760db35172a2224b731719ed6c76, L2StartNum: 2099776, L2EndNum: 2100073, L2BlockCnt, 298
[400/408] Channel ID: f8e001697b195b21e58a35126a52ec5b, L2StartNum: 2077784, L2EndNum: 2078085, L2BlockCnt, 302
[401/408] Channel ID: f9427495f0dbf967f238998c44011902, L2StartNum: 2124724, L2EndNum: 2125023, L2BlockCnt, 300
[402/408] Channel ID: f999b633449917a75d12a1236b697de0, L2StartNum: 2060554, L2EndNum: 2060845, L2BlockCnt, 292
[403/408] Channel ID: f9aa2069176be12edd4f34c6a863764b, L2StartNum: 2043099, L2EndNum: 2043390, L2BlockCnt, 292
[404/408] Channel ID: fae0d8a3afd0ae92befaebf716c3fd11, L2StartNum: 2070094, L2EndNum: 2070393, L2BlockCnt, 300
[405/408] Channel ID: fb44b33b51a977272b32d4fe0713f810, L2StartNum: 2087589, L2EndNum: 2087888, L2BlockCnt, 300
[406/408] Channel ID: fd22d73ca508673ef67453f79f518e7e, L2StartNum: 2014856, L2EndNum: 2015150, L2BlockCnt, 295
[407/408] Channel ID: fe37e62106e73584f7934c3c1201a6f5, L2StartNum: 2021379, L2EndNum: 2021678, L2BlockC

In [15]:
!TX_ENCODE_TYPE=2  && $SPAN_BATCH_TESTER convert --in=$DB_PREFIX/channel_cache --out=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --l2=$L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[399/408] Channel ID: f718760db35172a2224b731719ed6c76, L2StartNum: 2099776, L2EndNum: 2100073, L2BlockCnt, 298
[400/408] Channel ID: f8e001697b195b21e58a35126a52ec5b, L2StartNum: 2077784, L2EndNum: 2078085, L2BlockCnt, 302
[401/408] Channel ID: f9427495f0dbf967f238998c44011902, L2StartNum: 2124724, L2EndNum: 2125023, L2BlockCnt, 300
[402/408] Channel ID: f999b633449917a75d12a1236b697de0, L2StartNum: 2060554, L2EndNum: 2060845, L2BlockCnt, 292
[403/408] Channel ID: f9aa2069176be12edd4f34c6a863764b, L2StartNum: 2043099, L2EndNum: 2043390, L2BlockCnt, 292
[404/408] Channel ID: fae0d8a3afd0ae92befaebf716c3fd11, L2StartNum: 2070094, L2EndNum: 2070393, L2BlockCnt, 300
[405/408] Channel ID: fb44b33b51a977272b32d4fe0713f810, L2StartNum: 2087589, L2EndNum: 2087888, L2BlockCnt, 300
[406/408] Channel ID: fd22d73ca508673ef67453f79f518e7e, L2StartNum: 2014856, L2EndNum: 2015150, L2BlockCnt, 295
[407/408] Channel ID: fe37e62106e73584f7934c3c1201a6f5, L2StartNum: 2021379, L2EndNum: 2021678, L2BlockC

span batch tester `analyze`: Analyze channels with singluar batches by comparing with corresponding span batches.

In [16]:
!TX_ENCODE_TYPE=0 && $SPAN_BATCH_TESTER analyze --in-channel=$DB_PREFIX/channel_cache --in-span-batch=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --out=$DB_PREFIX/result_v$TX_ENCODE_TYPE --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[399/408] Channel ID: f718760db35172a2224b731719ed6c76, CompressedReductionPercent: 59.449730 %
[400/408] Channel ID: f8e001697b195b21e58a35126a52ec5b, CompressedReductionPercent: 87.811448 %
[401/408] Channel ID: f9427495f0dbf967f238998c44011902, CompressedReductionPercent: 99.220198 %
[402/408] Channel ID: f999b633449917a75d12a1236b697de0, CompressedReductionPercent: 72.608308 %
[403/408] Channel ID: f9aa2069176be12edd4f34c6a863764b, CompressedReductionPercent: 84.994978 %
[404/408] Channel ID: fae0d8a3afd0ae92befaebf716c3fd11, CompressedReductionPercent: 74.517266 %
[405/408] Channel ID: fb44b33b51a977272b32d4fe0713f810, CompressedReductionPercent: 68.855047 %
[406/408] Channel ID: fd22d73ca508673ef67453f79f518e7e, CompressedReductionPercent: 74.344655 %
[407/408] Channel ID: fe37e62106e73584f7934c3c1201a6f5, CompressedReductionPercent: 79.385007 %
[408/408] Channel ID: fec8a38f594dbce0c01c89dff3471f6d, CompressedReductionPercent: 96.903906 %


In [17]:
!TX_ENCODE_TYPE=1 && $SPAN_BATCH_TESTER analyze --in-channel=$DB_PREFIX/channel_cache --in-span-batch=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --out=$DB_PREFIX/result_v$TX_ENCODE_TYPE --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[399/408] Channel ID: f718760db35172a2224b731719ed6c76, CompressedReductionPercent: 58.549756 %
[400/408] Channel ID: f8e001697b195b21e58a35126a52ec5b, CompressedReductionPercent: 87.535354 %
[401/408] Channel ID: f9427495f0dbf967f238998c44011902, CompressedReductionPercent: 99.212477 %
[402/408] Channel ID: f999b633449917a75d12a1236b697de0, CompressedReductionPercent: 72.062590 %
[403/408] Channel ID: f9aa2069176be12edd4f34c6a863764b, CompressedReductionPercent: 84.780716 %
[404/408] Channel ID: fae0d8a3afd0ae92befaebf716c3fd11, CompressedReductionPercent: 74.091901 %
[405/408] Channel ID: fb44b33b51a977272b32d4fe0713f810, CompressedReductionPercent: 68.167465 %
[406/408] Channel ID: fd22d73ca508673ef67453f79f518e7e, CompressedReductionPercent: 73.843173 %
[407/408] Channel ID: fe37e62106e73584f7934c3c1201a6f5, CompressedReductionPercent: 79.075691 %
[408/408] Channel ID: fec8a38f594dbce0c01c89dff3471f6d, CompressedReductionPercent: 96.796881 %


In [18]:
!TX_ENCODE_TYPE=2 && $SPAN_BATCH_TESTER analyze --in-channel=$DB_PREFIX/channel_cache --in-span-batch=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --out=$DB_PREFIX/result_v$TX_ENCODE_TYPE --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[399/408] Channel ID: f718760db35172a2224b731719ed6c76, CompressedReductionPercent: 59.895432 %
[400/408] Channel ID: f8e001697b195b21e58a35126a52ec5b, CompressedReductionPercent: 88.060606 %
[401/408] Channel ID: f9427495f0dbf967f238998c44011902, CompressedReductionPercent: 99.220198 %
[402/408] Channel ID: f999b633449917a75d12a1236b697de0, CompressedReductionPercent: 72.814345 %
[403/408] Channel ID: f9aa2069176be12edd4f34c6a863764b, CompressedReductionPercent: 85.148979 %
[404/408] Channel ID: fae0d8a3afd0ae92befaebf716c3fd11, CompressedReductionPercent: 74.797112 %
[405/408] Channel ID: fb44b33b51a977272b32d4fe0713f810, CompressedReductionPercent: 69.247224 %
[406/408] Channel ID: fd22d73ca508673ef67453f79f518e7e, CompressedReductionPercent: 74.686574 %
[407/408] Channel ID: fe37e62106e73584f7934c3c1201a6f5, CompressedReductionPercent: 79.566958 %
[408/408] Channel ID: fec8a38f594dbce0c01c89dff3471f6d, CompressedReductionPercent: 97.010932 %


span batch tester `fetch`: Fetch consecutive singular batches.

In [19]:
!$SPAN_BATCH_TESTER fetch --l2 $L2_RPC --start=$ACC_L2_START --end=$ACC_L2_END --concurrent-requests=100 --out=$DB_PREFIX/singular_batches_cache | tail -10

Fetched L2 block:  2142992
Fetched L2 block:  2142993
Fetched L2 block:  2142995
Fetched L2 block:  2142994
Fetched L2 block:  2142998
Fetched L2 block:  2142997
Fetched L2 block:  2142996
Fetched L2 block:  2142999
Fetched v0 batches in range [2142000,2143000).
Fetch Config: Chain ID: 424.


span batch tester `merge`: Merge singular batches:

This command will analyze `start - end` number of batches, Comparing uncompressed size of below two batches:
- For `i` in range  `0` to `start - end - 1`
  - singular batch from number `start` to `start + i`, inclusive.
  - span batch from number `start` to `start + i`, inclusive.

Upper command will make a result json file which has name of the form `{start_number}_{end_number}.json`.

In [12]:
!$SPAN_BATCH_TESTER merge --start=$ACC_L2_START --end=$ACC_L2_END --l2 $L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --in=$DB_PREFIX/singular_batches_cache --out=$DB_PREFIX/merge_result | tail -10

Merged v0 batches in range [2142000,2143000).


span batch tester `format`: Compare different tx encodings for span batch.


In [21]:
!$SPAN_BATCH_TESTER format --permutation 0123465 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: 0.000000 %
[405/408] cnt: 377, delta: 0, deltasum: 57, originalCompressedSizeSum: 1381358
Reduction Percentage: 0.000000 %
[406/408] cnt: 378, delta: 1, deltasum: 58, originalCompressedSizeSum: 1385799
Reduction Percentage: 0.022517 %
[407/408] cnt: 379, delta: 5, deltasum: 63, originalCompressedSizeSum: 1389168
Reduction Percentage: 0.148412 %
[408/408] cnt: 380, delta: 0, deltasum: 63, originalCompressedSizeSum: 1389559
Reduction Percentage: 0.000000 %
Final Reduction Percentage: 0.004534 %


In [22]:
!$SPAN_BATCH_TESTER format --permutation 0134625 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: -0.022207 %
[405/408] cnt: 388, delta: 2, deltasum: 452, originalCompressedSizeSum: 1381358
Reduction Percentage: 0.033129 %
[406/408] cnt: 389, delta: 1, deltasum: 453, originalCompressedSizeSum: 1385799
Reduction Percentage: 0.022517 %
[407/408] cnt: 390, delta: 1, deltasum: 454, originalCompressedSizeSum: 1389168
Reduction Percentage: 0.029682 %
[408/408] cnt: 391, delta: 0, deltasum: 454, originalCompressedSizeSum: 1389559
Reduction Percentage: 0.000000 %
Final Reduction Percentage: 0.032672 %


In [23]:
!$SPAN_BATCH_TESTER format --permutation 3041265 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: 0.000000 %
[405/408] cnt: 319, delta: 1, deltasum: 357, originalCompressedSizeSum: 1381358
Reduction Percentage: 0.016565 %
[406/408] cnt: 320, delta: 3, deltasum: 360, originalCompressedSizeSum: 1385799
Reduction Percentage: 0.067552 %
[407/408] cnt: 321, delta: 8, deltasum: 368, originalCompressedSizeSum: 1389168
Reduction Percentage: 0.237459 %
[408/408] cnt: 321, delta: -2, deltasum: 366, originalCompressedSizeSum: 1389559
Reduction Percentage: -0.511509 %
Final Reduction Percentage: 0.026339 %


In [24]:
!$SPAN_BATCH_TESTER format --permutation 0125634 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: 0.000000 %
[405/408] cnt: 326, delta: -1, deltasum: 77, originalCompressedSizeSum: 1381358
Reduction Percentage: -0.016565 %
[406/408] cnt: 326, delta: -1, deltasum: 76, originalCompressedSizeSum: 1385799
Reduction Percentage: -0.022517 %
[407/408] cnt: 327, delta: 0, deltasum: 76, originalCompressedSizeSum: 1389168
Reduction Percentage: 0.000000 %
[408/408] cnt: 328, delta: 0, deltasum: 76, originalCompressedSizeSum: 1389559
Reduction Percentage: 0.000000 %
Final Reduction Percentage: 0.005469 %


In [25]:
!$SPAN_BATCH_TESTER format --permutation 0324165 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: 0.044415 %
[405/408] cnt: 379, delta: 2, deltasum: 587, originalCompressedSizeSum: 1381358
Reduction Percentage: 0.033129 %
[406/408] cnt: 380, delta: 3, deltasum: 590, originalCompressedSizeSum: 1385799
Reduction Percentage: 0.067552 %
[407/408] cnt: 381, delta: 6, deltasum: 596, originalCompressedSizeSum: 1389168
Reduction Percentage: 0.178094 %
[408/408] cnt: 381, delta: -1, deltasum: 595, originalCompressedSizeSum: 1389559
Reduction Percentage: -0.255754 %
Final Reduction Percentage: 0.042819 %


In [26]:
!$SPAN_BATCH_TESTER format --permutation 3041256 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: 0.000000 %
[405/408] cnt: 306, delta: 1, deltasum: 287, originalCompressedSizeSum: 1381358
Reduction Percentage: 0.016565 %
[406/408] cnt: 307, delta: 2, deltasum: 289, originalCompressedSizeSum: 1385799
Reduction Percentage: 0.045035 %
[407/408] cnt: 308, delta: 7, deltasum: 296, originalCompressedSizeSum: 1389168
Reduction Percentage: 0.207777 %
[408/408] cnt: 308, delta: -2, deltasum: 294, originalCompressedSizeSum: 1389559
Reduction Percentage: -0.511509 %
Final Reduction Percentage: 0.021158 %


In [27]:
!$SPAN_BATCH_TESTER format --permutation 3401256 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: -0.022207 %
[405/408] cnt: 233, delta: -2, deltasum: -94, originalCompressedSizeSum: 1381358
Reduction Percentage: -0.033129 %
[406/408] cnt: 234, delta: 0, deltasum: -94, originalCompressedSizeSum: 1385799
Reduction Percentage: 0.000000 %
[407/408] cnt: 235, delta: 4, deltasum: -90, originalCompressedSizeSum: 1389168
Reduction Percentage: 0.118730 %
[408/408] cnt: 235, delta: -2, deltasum: -92, originalCompressedSizeSum: 1389559
Reduction Percentage: -0.511509 %
Final Reduction Percentage: -0.006621 %


In [28]:
!$SPAN_BATCH_TESTER format --permutation 6051234 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: -0.044415 %
[405/408] cnt: 199, delta: -2, deltasum: -345, originalCompressedSizeSum: 1381358
Reduction Percentage: -0.033129 %
[406/408] cnt: 199, delta: -6, deltasum: -351, originalCompressedSizeSum: 1385799
Reduction Percentage: -0.135105 %
[407/408] cnt: 200, delta: 1, deltasum: -350, originalCompressedSizeSum: 1389168
Reduction Percentage: 0.029682 %
[408/408] cnt: 200, delta: -2, deltasum: -352, originalCompressedSizeSum: 1389559
Reduction Percentage: -0.511509 %
Final Reduction Percentage: -0.025332 %


In [29]:
!$SPAN_BATCH_TESTER format --permutation 0132456 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: 0.000000 %
[405/408] cnt: 384, delta: 1, deltasum: 284, originalCompressedSizeSum: 1381358
Reduction Percentage: 0.016565 %
[406/408] cnt: 385, delta: 1, deltasum: 285, originalCompressedSizeSum: 1385799
Reduction Percentage: 0.022517 %
[407/408] cnt: 386, delta: 5, deltasum: 290, originalCompressedSizeSum: 1389168
Reduction Percentage: 0.148412 %
[408/408] cnt: 387, delta: 0, deltasum: 290, originalCompressedSizeSum: 1389559
Reduction Percentage: 0.000000 %
Final Reduction Percentage: 0.020870 %


In [30]:
!$SPAN_BATCH_TESTER format --permutation 0352614 --in-span-batch=$DB_PREFIX/span_batch_cache_v2 --out=$DB_PREFIX/format_result --chain-id=$L2_CHAIN_ID | tail -10

Reduction Percentage: 0.044415 %
[405/408] cnt: 379, delta: 2, deltasum: 557, originalCompressedSizeSum: 1381358
Reduction Percentage: 0.033129 %
[406/408] cnt: 380, delta: 0, deltasum: 557, originalCompressedSizeSum: 1385799
Reduction Percentage: 0.000000 %
[407/408] cnt: 381, delta: 6, deltasum: 563, originalCompressedSizeSum: 1389168
Reduction Percentage: 0.178094 %
[408/408] cnt: 381, delta: -1, deltasum: 562, originalCompressedSizeSum: 1389559
Reduction Percentage: -0.255754 %
Final Reduction Percentage: 0.040444 %
